In [2]:
!pip install pdfplumber


[notice] A new release of pip is available: 24.1.2 -> 25.1.1
[notice] To update, run: C:\Users\aashi\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [17]:
import pdfplumber
import re
import pandas as pd

# Load PDF
pdf_path = input("Enter the path to your blood report PDF: ").strip()

extracted_text = ""
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        extracted_text += page.extract_text() + "\n"


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

In [4]:
# Regex to extract tests in the format: Test_name value range unit"
pattern= re.compile(
    r'(?P<test>[A-Z0-9 /()-]{3,})\s+(?P<value>[0-9.]+)\s+(?P<range>[<>=/\s0-9.-]+)?\s*(?P<unit>[a-zA-Z/%µμ.]+)?',
    re.IGNORECASE
)

matches= pattern.finditer(extracted_text)

results=[]
for match in matches:
    test= match.group("test").strip()
    value= match.group("value")
    ref_range = match.group("range").strip() if match.group("range") else "-"
    unit = match.group("unit").strip() if match.group("unit") else "-"

    results.append({
        "Test": test,
        "Result": value,
        "Reference Range": ref_range,
        "Unit": unit
    })

df = pd.DataFrame(results)

In [5]:
# Show only some common tests
important_tests = df[df['Test'].str.contains("HEMOGLOBIN|GLUCOSE|T3|T4|TSH|VITAMIN D|IRON|CHOLESTEROL", case=False)]

# Display clean table
important_tests.reset_index(drop=True, inplace=True)
important_tests


,Test,Result,Reference Range,Unit
0,HEMOGLOBIN (HB),14.5,13.0 - 17.0,g/dL
1,MEAN CORPUSCULAR HEMOGLOBIN (MCH),28.9,27.0 - 32.0,pg
2,MEAN CORPUSCULAR HEMOGLOBIN,32.9,31.5 - 34.5,g/dL
3,ESTIMATED AVERAGE GLUCOSE(EAG),116.9,-,High
4,HDL CHOLESTEROL,36,-,Low
5,CHOLESTEROL LDL,107,-,High
6,NON HDL CHOLESTEROL,153,-,High
7,IRON 73 65 -,175,-,µg/dL
8,TOTAL IRON BINDING CAPACITY 321 250 -,450,-,µg/dL
9,TSH (ULTRASENSITIVE),2.542,0.550 - 4.780,µIU/mL


In [6]:
def get_flag(value, ref_range):
    try:
        val = float(value)
        nums = re.findall(r'\d+\.?\d*', ref_range)
        if len(nums) == 2:
            low, high = float(nums[0]), float(nums[1])
            if val < low:
                return "Low"
            elif val > high:
                return "High"
            else:
                return "Normal"
    except:
        return "Unknown"
    return "Unknown"

df["Flag"] = df.apply(lambda row: get_flag(row["Result"], row["Reference Range"]), axis=1)
df[df["Flag"] != "Normal"]


,Test,Result,Reference Range,Unit,Flag
0,Amritsar,143001,-,ABHA,Unknown
3,WHITE BLOOD CELL (WBC) COUNT,14.10,-,High,Unknown
4,0 -,10.0,-,thou/µL,Unknown
5,PLATELET COUNT 296 150 -,410,-,thou/µL,Unknown
10,RED CELL DISTRIBUTION WIDTH (RDW),14.3,-,High,Unknown
...,...,...,...,...,...
126,clinical,2.,-,All,Unknown
127,Incorrect specimen type,8.,-,Test,Unknown
128,Discrepancy between identification on specimen,9.,-,Incaseofqueries,Unknown
129,Mohali,160062,-,Page,Unknown


In [7]:
insights = []

if (
    float(df[df["Test"].str.contains("GLUCOSE", case=False)]["Result"].values[0]) > 100 and
    float(df[df["Test"].str.contains("TRIGLYCERIDES", case=False)]["Result"].values[0]) > 150
):
    insights.append(" Possible insulin resistance: High glucose + high triglycerides")

if (
    float(df[df["Test"].str.contains("HEMOGLOBIN", case=False)]["Result"].values[0]) < 13 and
    float(df[df["Test"].str.contains("IRON", case=False)]["Result"].values[0]) < 65
):
    insights.append(" Possible anemia: Low hemoglobin and iron")

if insights:
    for i in insights:
        print(i)
else:
    print(" No second-order patterns detected.")


 Possible insulin resistance: High glucose + high triglycerides


In [8]:
import json

# First-order findings
first_order = df[df["Flag"] != "Normal"]
first_order_dict = dict(zip(first_order["Test"], first_order["Flag"]))

# Second-order insight list 
second_order = insights

# Manually write causal hypotheses
causal_hypotheses = []
if "glucose" in df["Test"].str.lower().values and "triglycerides" in df["Test"].str.lower().values:
    causal_hypotheses.append("High triglycerides + glucose suggest poor glucose metabolism")

if "vitamin d" in df["Test"].str.lower().values:
    vit_d_val = float(df[df["Test"].str.contains("VITAMIN D", case=False)]["Result"].values[0])
    if vit_d_val < 20:
        causal_hypotheses.append("Low Vitamin D may reduce insulin sensitivity")

# Basic narrative summary (manual without GPT)
narrative = "Based on the blood report, the patient shows: " + (
    ", ".join([f"{k} is {v}" for k, v in first_order_dict.items()])
    + ". " + " ".join(second_order)
    if second_order else "mostly normal values."
)

# Final output dict
final_output = {
    "FirstOrderFindings": first_order_dict,
    "SecondOrderInsights": second_order,
    "CausalHypotheses": causal_hypotheses,
    "NarrativeExplanation": narrative
}

# Show nicely
print(json.dumps(final_output, indent=4))


{
    "FirstOrderFindings": {
        "Amritsar": "Unknown",
        "WHITE BLOOD CELL (WBC) COUNT": "Unknown",
        "0 -": "Unknown",
        "PLATELET COUNT 296 150 -": "Unknown",
        "RED CELL DISTRIBUTION WIDTH (RDW)": "Unknown",
        "6 -": "Unknown",
        "MENTZER INDEX": "Unknown",
        "NEUTROPHILS 62 40 -": "Unknown",
        "LYMPHOCYTES 32 20 -": "Unknown",
        "MONOCYTES 4 2 -": "Unknown",
        "EOSINOPHILS 2 1 -": "Unknown",
        "Page 1 Of": "Unknown",
        "BASOPHILS 0 0 -": "Unknown",
        "ABSOLUTE NEUTROPHIL COUNT": "Unknown",
        "ABSOLUTE LYMPHOCYTE COUNT": "Unknown",
        "ABSOLUTE BASOPHIL COUNT": "Unknown",
        "02 -": "Unknown",
        "NEUTROPHIL LYMPHOCYTE RATIO (NLR)": "Unknown",
        "Page 2 Of": "Unknown",
        "R 03 0 - 14 mm at": "Unknown",
        "HBA1C": "Unknown",
        "7 -": "Unknown",
        "ESTIMATED AVERAGE GLUCOSE(EAG)": "Unknown",
        "Page 3 Of": "Unknown",
        "Page 4 Of": "Unknown

In [9]:
with open("blood_report_insights.json", "w") as f:
    json.dump(final_output, f, indent=4)